In [10]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 500

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%%sql --save first_contacts --no-execute

SELECT target_object_id, first_description, timestamp
FROM (
    SELECT
    target_object_id,
    timestamp,
    first_value(description) OVER (PARTITION BY target_object_id ORDER BY timestamp) first_description
    FROM actstream_action
    WHERE verb='Contacted complainant:'
) first_descriptions
WHERE first_description NOT LIKE '%CRT Auto response%'
GROUP BY 1, 2, 3

There's a new jupysql version available (0.7.9), you're running 0.7.8. To upgrade: pip install jupysql --upgrade


Skipping execution...

In [12]:
%%sql --save letter_kinds --no-execute

WITH languages (name) as (
    values
        ('Spanish'),
        ('English'),
        ('Chinese Traditional'),
        ('Chinese Simplified'),
        ('Vietnamese'),
        ('Korean'),
        ('Tagalog')
)
SELECT
    title,
    COALESCE((SELECT
        TRIM(REPLACE(title, '(' || languages.name || ')', ''))
        FROM languages
        WHERE title LIKE '%' || languages.name || '%'
        LIMIT 1
    ), title) AS clean_title,
    language,
    referral_contact_id IS NOT NULL AS is_referral,
    title LIKE '%Constant Writer%' AS is_constant_writer,
    title LIKE '%No Capacity%' AS is_no_capacity
FROM cts_forms_responsetemplate

Skipping execution...

In [13]:
%%sql --save day_buckets --no-execute

WITH day_buckets (position, low, high, name) AS (
    VALUES
        (0, INTERVAL '0 days', INTERVAL '1 day', 'Less than a day'),
        (1, INTERVAL '1 day', INTERVAL '2 days', 'Two days'),
        (2, INTERVAL '2 days', '3 days', 'Three days'),
        (3, INTERVAL '3 days', '4 days', 'Four days'),
        (4, INTERVAL '4 days', '5 days', 'Five days'),
        (5, INTERVAL '5 days', '6 days', 'Six days'),
        (6, INTERVAL '6 days', '7 days', 'One week'),
        (7, INTERVAL '8 days', '14 days', 'Two Weeks'),
        (8, INTERVAL '15 days', '31 days', 'One Month'),
        (9, INTERVAL '31 days', '1000 years', 'More than a Month')
) SELECT * FROM day_buckets

Skipping execution...

In [14]:
%%sql response_times << --with letter_kinds --with first_contacts --with day_buckets

SELECT
    day_buckets.position,
    day_buckets.name time_to_response,
    COALESCE(letter_kinds.clean_title, first_contacts.first_description) title,
    letter_kinds.language,
    letter_kinds.is_referral,
    letter_kinds.is_constant_writer,
    letter_kinds.is_no_capacity,
    COUNT(*) responses_sent
FROM first_contacts
LEFT JOIN letter_kinds ON first_description LIKE '%' || letter_kinds.title || '%'
LEFT JOIN cts_forms_report ON CAST(cts_forms_report.id AS varchar) = target_object_id
LEFT JOIN day_buckets ON
    day_buckets.low < first_contacts.timestamp - cts_forms_report.create_date AND
    day_buckets.high >= first_contacts.timestamp - cts_forms_report.create_date
GROUP BY 1,2,3,4,5,6,7
ORDER BY position

In [15]:
import plotly.express as px
df = response_times.DataFrame()

In [41]:
language_df = df.groupby(["is_referral", "language", "time_to_response"], as_index=False)['responses_sent'].sum()

fig = px.bar(
    language_df,
    x="time_to_response",
    y="responses_sent",
    color="language",
    facet_col="is_referral",
    category_orders={
        "time_to_response": ["Less than a day", "Two days", "Three days", "Four days", "Five days", "Six days", "One week", "Two Weeks", "One Month", "More than a Month",
],
        "is_referral": [True, False],
        "language": ["en", "es", "zh-hant", "zh-hans", "vi", "ko", "tl"]
    },
    height=700,
)

fig.show()

In [37]:
title_df = df.loc[df['is_referral'] == False]
title_df = title_df.groupby(["title", "time_to_response"], as_index=False)['responses_sent'].sum()

fig = px.bar(
    title_df,
    x="time_to_response",
    y="responses_sent",
    color="title",
    category_orders={
        "time_to_response": ["Less than a day", "Two days", "Three days", "Four days", "Five days", "Six days", "One week", "Two Weeks", "One Month", "More than a Month",
],
    },
    height=700,
)

fig.show()

In [38]:
title_df = df.loc[df['is_referral'] == True]
title_df = title_df.groupby(["time_to_response", "title", ], as_index=False)['responses_sent'].sum()

fig = px.bar(
    title_df,
    x="time_to_response",
    y="responses_sent",
    color="title",
    category_orders={
        "time_to_response": ["Less than a day", "Two days", "Three days", "Four days", "Five days", "Six days", "One week", "Two Weeks", "One Month", "More than a Month",
],
    },
    height=700,
)

fig.show()